In [6]:
import os
import ee
import folium

In [7]:
ee.Initialize()

In [8]:
import geemap

In [ ]:
from ipynb.fs.full.input1 import *

In [52]:
from ipynb.fs.defs.input1 import *

In [ ]:
# run input1.ipynb

In [9]:
iDate = '2021-01-01'
fDate = '2022-01-01'

In [ ]:
geom = ee.FeatureCollection([
    ee.Feature(
        ee.Geometry.Polygon([[
            [48.8143604764154, -13.233302999964707],
            [48.8143604764154, -13.300135050067755],
            [48.9242237576654, -13.300135050067755],
            [48.9242237576654, -13.233302999964707]
        ]]),
        {'system:index': '0'}
    )
])

In [17]:
geom = ee.Geometry.Rectangle([[48.4684, -12.6752], [48.9465, -13.6273]])

In [23]:
geom2 = ee.Geometry(geom).buffer(10000)

In [ ]:
roi = ee.FeatureCollection("projects/ee-pierdubo/assets/roi_nw_smooth")

In [44]:
nw_2box = ee.FeatureCollection('projects/ee-pierdubo/assets/nw_2box')

In [50]:
aoi = (nw_2box
       .filter(ee.Filter.eq('sel', 1)))

In [9]:
targetBands = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','NDVI','NBR','EVI','BRIGHTNESS','GREENNESS','WETNESS']

In [12]:
lsColl = getLandsat(iDate, fDate, geom, targetBands, 2)

In [ ]:
Map = geemap.Map(center=(-13.5, 48), zoom = 8)
Map.addLayer(nw2, {}, 'Geom')
Map

In [14]:
Map.addLayer(lsColl.select(['SWIR1','NIR','GREEN']).median(), {'bands': ['SWIR1','NIR','GREEN'], 'min':0.020, 'max':0.400, 'gamma': 0.85}, 'Landsat Coll')

In [ ]:
aoi = ee.Geometry.LineString(
    [[49.09178474992574, -11.924273142945154],
     [48.66331795305074, -13.275179628809104],
     [47.818428215464124, -14.41247011042455],
     [48.179964869881715, -14.846057452723358],
     [48.08998151060017, -15.275048458710872],
     [47.753046040851736, -14.483073303752759],
     [46.60887459367574, -15.668473695347696],
     [44.40062264055074, -16.196686141581498]])
# AOI for upper area, using a polyline

In [6]:
geom = ee.FeatureCollection([
    ee.Feature(
        ee.Geometry.Polygon([[
            [48.8143604764154, -13.233302999964707],
            [48.8143604764154, -13.300135050067755],
            [48.9242237576654, -13.300135050067755],
            [48.9242237576654, -13.233302999964707]
        ]]),
        {'system:index': '0'}
    )
])

In [ ]:
geometry3 = ee.Geometry.Polygon(
        [[[48.93600246152643, -12.96147823789479],
          [48.93600246152643, -12.979711921622673],
          [48.957803456399475, -12.979711921622673],
          [48.957803456399475, -12.96147823789479]]], {}, False)
# displayProperties: [{"type": "rectangle"}] #

In [38]:
tccCollection = ee.ImageCollection("projects/servir-mekong/UMD/tree_canopy")
rubberData = ee.FeatureCollection("projects/servir-mekong/referenceData/CambodiaRubber")
trainingSample = ee.FeatureCollection("projects/servir-mekong/ReferenceData/EvergreenTraining")

In [63]:
collect_S2 = (ee.ImageCollection('COPERNICUS/S2')
              .filter(ee.Filter.calendarRange(2020,2020,'YEAR'))
              .filter(ee.Filter.calendarRange(1,12,'MONTH'))
              .filter(ee.Filter.calendarRange(1,31, 'DAY_OF_MONTH'))
              .filterBounds(aoi)
              .map(maskS2clouds) 
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25)))
              # .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 75)

In [ ]:
collect_VV = ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filter(ee.Filter.calendarRange(2020,2020,'YEAR')) \
  .filter(ee.Filter.calendarRange(1,12,'MONTH')) \
  .filter(ee.Filter.calendarRange(1,31, 'DAY_OF_MONTH')) \
  .filter(ee.Filter.eq('instrumentMode', 'IW')) \
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
  .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
  .filterMetadata('resolution_meters', 'equals' , 10) \
  .filterBounds(aoi) \
  .select('VV')

collect_VH = ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filter(ee.Filter.calendarRange(2020,2020,'YEAR')) \
  .filter(ee.Filter.calendarRange(1,12,'MONTH')) \
  .filter(ee.Filter.calendarRange(1,31, 'DAY_OF_MONTH')) \
  .filter(ee.Filter.eq('instrumentMode', 'IW')) \
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
  .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
  .filterMetadata('resolution_meters', 'equals' , 10) \
  .filterBounds(aoi) \
  .select('VH')

In [ ]:
#print(collect_S2, 'S2 catalogue')
#print(collect_VH, collect_VV, 'VH & VV catalogue')

In [ ]:
y1 = 2020
start_S1 = ee.Date.fromYMD(y1,1,1)
end_S1 = ee.Date.fromYMD(y1,12,31)

Filter by date; apply speckle filter ... also see addS1 function

In [ ]:
SARVV = collect_VV.filterDate(start_S1,end_S1).mosaic()
SARVH = collect_VH.filterDate(start_S1,end_S1).mosaic()

In [ ]:
SMOOTHING_RADIUS = 30
SARVV_flt = SARVV.select('VV').focal_mean(SMOOTHING_RADIUS, 'circle', 'meters')

In [ ]:
timeField = 'system:time_start'

In [ ]:
y2 = 2021

start = ee.Date.fromYMD(2015,1,1)
end = ee.Date.fromYMD(y2,4,1)

S2_comp = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate(start,end) \
  .filterBounds(aoi) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) \
  .map(maskS2clouds)

In [62]:
Map.addLayer(collect_S2.select(['B11','B8','B3']).median(), {'bands': ['B11','B8','B3'], 'min':200, 'max':4000, 'gamma': 0.85}, 'Sentinel-2 Coll')

In [61]:
Map = geemap.Map(center=(-13.5, 48), zoom = 8)
Map

Map(center=[-13.5, 48], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

In [ ]:
Map.addLayer(lsColl.select(['SWIR1','NIR','GREEN']).median(), {'bands': ['SWIR1','NIR','GREEN'], 'min':0.020, 'max':0.400, 'gamma': 0.85}, 'Landsat Coll')

In [ ]:
Map.draw_features
roi = ee.FeatureCollection(Map.draw_features)
roi

In [ ]:
Map.addLayer(srtm, dem_vis, 'SRTM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover', 0)
Map.addLayer(L8_collect, landsat_vis, 'Landsat 8', 0)
Map.addLayer(SARVV, {'min':-15, 'max':0}, 'SAR VV', 0)
Map.addLayer(SARVH, {'min':-25, 'max':0}, 'SAR VH', 0)
Map.addLayer(collect_S2, {'bands': ['B4',  'B3',  'B2'], 'min':200, 'max': 4000}, 'S2 Catalogue', 0)
Map.addLayer(S2_comp, {'bands': ['B4',  'B3',  'B2'], 'min':200, 'max':4000, 'gamma': 0.85}, "S2 Comp", 1)

Map

In [ ]:
def addS2var(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8A', 'B4']).rename('NDVI')).float() \
    .addBands(ee.Image(years).rename('t').float()).addBands(ee.Image.constant(1))

Remove clouds, add variables and filter to the area of interest

In [ ]:
filteredS2 = (S2_comp
              .filterBounds(aoi)
              .map(maskS2clouds)
              .map(addS2var))

Plot a time series of NDVI at a single location.

In [ ]:
#S2Chart = ui.Chart.image.series(filteredS2.select('NDVI'), geometry3) \
#    .setChartType('ScatterChart') \
#    .setOptions({ \
#      'title': 'S2 NDVI time series at ROI', \
#      'lineWidth': 1, \
#      'pointSize': 3, \
#    })
#print(S2Chart)

#print(filteredS2, "List of S2 images at loc")

In [ ]:
dayOfInterestS2 = ee.Date('2016-10-23')

S2doi = ee.ImageCollection(S2_comp) \
    .filterDate(dayOfInterestS2, dayOfInterestS2.advance(1, 'day')) \
    .filterBounds(aoi) \
    .map(maskS2clouds)

In [ ]:
Map = geemap.Map(center=(-13, 48.8), zoom = 7)
Map.addLayer(srtm, dem_vis, 'SRTM DEM', True, 0.5)

Map.addLayer(filteredS2, {'bands': ['B4',  'B3',  'B2'], 'min':400, 'max':3000, 'gamma': 0.85}, "S2 NDVI", 1)
Map.addLayer(S2doi, {'bands': ['B4',  'B3',  'B2'], 'min':400, 'max':3000, 'gamma': 0.85}, "S2 DOI", 1)
Map.addLayer(magr, {'color': "FF0000"}, "GMW 2016", 0)
Map

### Folium

In [9]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        constrol=True
    ).add_to(self)

folium.map.add_ee_layer = add_ee_layer    

In [21]:
map = folium.Map(location=[-13.8, 48.3], tiles='OpenStreetMap', zoom_start=8)

In [ ]:
# map.add_ee_layer(srtm.select('elevation'), {'min':0, 'max': 1200}, 'NASADEM')
map.add_ee_layer(Mangr, {}, 'Mangrove')

# map.add_ee_layer(lc_esa, lcEsa_vis_params , 'ESA Land Cover')
# map.add_ee_layer(PALSAR_2017.select('HV'), {'min':-0, 'max': 10000}, 'Palsar 2017')
# map.add_ee_layer(L4_coll.select('B4').median(), {}, 'L4 Coll')
# map.add_ee_layer(collection7.select(['SWIR1','NIR','GREEN']).median(), {'bands': ['SWIR1','NIR','GREEN'], 'min':0.020, 'max':0.400, 'gamma': 0.85}, 'Landsat Coll')
# mp.add_ee_layer(indices.select('EVI2').median(), {}, 'NDVI L8 2021')

map.add_child(folium.LayerControl())
map

In [ ]:
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})

In [ ]:
map = folium.Map(location=[-13.8, 48.3], tiles='OpenStreetMap', zoom_start=8)

In [ ]:
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)

In [ ]:
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})

In [ ]:
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='predictions',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
# Arrange layers inside a list (elevation, LST and land cover).
ee_tiles = [srtm, landcover]

# Arrange visualization paramaters inside a list
ee_vis_params = [elv_vis_params, lc_vis_params]

# Arrange layer names inside a list.
ee_tiles_names = ['Elevation', 'Land Cover']

In [ ]:
'''

In [ ]:
Map = folium.Map(location=[-13.8, 48.3], tiles='OpenStreetMap', zoom_start=8)

# Add Layers to the map using a loop.
for tile, vis_param, name in zip(ee_tiles, ee_vis_params, ee_tiles_names):
    Map.add_ee_layer(tile, vis_param, name)

folium.LayerControl(collapsed = False).add_to(Map)
Map

In [ ]:
# display(Map)